In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [52]:
from szp_funcs import print_df, load_month, path_to_data

In [39]:
month = 'may'
mesh_const = 10000
path_to_file = 'C:/Users/PetukhovMD/Desktop/mesh_data.xlsx'
path_to_file_new = 'C:/Users/PetukhovMD/Desktop/post_check.xlsx'

In [17]:
dirty_may = pd.read_excel('C:\\Users\\PetukhovMD\\Desktop\\2022\\old май\\ДОНМ_Начисления работников.xlsx')

In [18]:
dirty_may.columns

Index(['КодЕКИС', 'Организация', 'ОрганизацияИНН', 'Период', 'СНИЛС', 'ФИО',
       'ДатаПриема', 'ДатаУвольнения', 'ДолжностьНаКонецМес',
       'ВидЗанятостиНаКонецМес', 'КоличествоСтавокНаКонецМес',
       'СостояниеКонецМес', 'ВсегоНачислено',
       'ВсегоНачисленоСучетомРаспределенияОтпуска',
       'НачисленоОтпускныхЗаТекущийМесяц', 'ВидЗанятостиНачалоМес',
       'КоличествоСтавокНачалоМес', 'СостояниеНачалоМес', 'ДолжностьНачалоМес',
       'ОтработаноДней', 'НормаДней', 'NumerRegion', 'NameBase', 'Abonent'],
      dtype='object')

In [20]:
cleaner = dirty_may.rename(columns={'ОрганизацияИНН': 'inn', 'СНИЛС': 'snils', 'ФИО': 'fio'})[['inn', 'snils', 'fio']]

In [4]:
res = load_month(month)

In [45]:
jobs = ["Заместитель директора","Заместитель начальника - начальник управления","Проректор","Воспитатель","Мастер производственного обучения","Учитель","Помощник воспитателя","Младший воспитатель","Ведущий специалист","Заведующий практикой","Заместитель начальника отдела","Заместитель начальника службы","Начальник группы","Начальник отдела","Начальник сектора","Начальник службы","Начальник управления","Специалист"]

In [48]:
res[res.job_may.isin(jobs)]['snils'].nunique()

107996

In [53]:
res2 = pd.read_excel(path_to_data + 'may.xlsx')

In [55]:
res2.type.unique()

array(['Основное место работы', 'Внешнее совместительство',
       'Внутреннее совместительство'], dtype=object)

In [64]:
res2.status.unique()

array(['Работа', nan, 'Отпуск основной',
       'Отпуск неоплачиваемый по разрешению работодателя', 'Увольнение',
       'Отпуск по уходу за ребенком', 'Отпуск по беременности и родам',
       'Работа в отпуске по уходу за ребенком', 'Болезнь',
       'Болезнь без оплаты', 'Отпуск учебный оплачиваемый',
       'Командировка', 'Отсутствие по невыясненным причинам',
       'Отсутствие с сохранением оплаты', 'Дополнительный отпуск',
       'Отпуск учебный неоплачиваемый',
       'Дополнительный отпуск (неоплачиваемый)',
       'Отпуск на санаторно-курортное лечение',
       'Отпуск неоплачиваемый по законодательству'], dtype=object)

In [56]:
res2.columns

Index(['inn', 'snils', 'sum', 'job', 'day', 'type', 'stv', 'status',
       'status_pref', 'wday', 'nday'],
      dtype='object')

In [61]:
a = res2[['inn', 'snils', 'type']][(res2.type == 'Внутреннее совместительство') | (res2.type == 'Внешнее совместительство') & (res2.status != 'Увольнение')].groupby(['inn', 'snils']).count()

In [63]:
print_df(a[a.type > 1].reset_index(), 'Внешники с внутренним')

In [40]:
post_ckeck = pd.read_excel(path_to_file_new)

In [41]:
post_info = pd.merge(post_ckeck, res[['inn', 'snils', 'sum_' + month, 'wday_' + month, 'nday_' + month, 'stv_' + month, 'job_' + month]], how='left')

In [42]:
post_info['true_sum'] = post_info.apply(lambda row: give_true_sum(row), axis=1)

In [44]:
print_df(post_info[post_info.true_sum < 94100], '84- без мэш post')

In [7]:
res

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may
0,5003021368,001-171-362 75,58500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
1,5003021368,001-191-281 85,112184.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
2,5003021368,001-244-107 65,118681.78,Учитель,0.0,Основное место работы,1.0,Работа,Работа,15.0,18.0
3,5003021368,001-263-234 77,79619.03,Воспитатель,0.0,Основное место работы,1.0,Отпуск основной,Работа,16.0,18.0
4,5003021368,001-322-594 80,37907.50,Специалист,0.0,Основное место работы,1.0,Работа,Работа,9.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
188387,9718071371,198-317-420 02,110851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188388,9718071371,199-418-719 35,153851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188389,9718071371,205-412-630 13,209199.25,Начальник группы,0.0,Основное место работы,1.0,Работа,Работа,13.0,18.0
188390,9718071371,206-135-584 35,27528.33,Бухгалтер,0.0,Основное место работы,1.0,NaN,NaN,6.0,11.0


In [6]:
mesh_data = pd.read_excel(path_to_file)

In [22]:
search = mesh_data.dropna()[['inn', 'fio']]

In [27]:
info = pd.merge(search, cleaner.drop_duplicates(), how='left', on=['inn', 'fio']).drop_duplicates()

In [28]:
info = pd.merge(info, res[['inn', 'snils', 'sum_' + month, 'wday_' + month, 'nday_' + month, 'stv_' + month, 'job_' + month]], how='left')

In [31]:
info

,inn,fio,snils,sum_may,wday_may,nday_may,stv_may,job_may
0,7717666051,Алимова Елена Владимировна,114-535-773 47,174937.67,23.0,23.0,1.0,Учитель
1,7717666051,Архипова Яна Николаевна,130-996-942 92,121383.16,23.0,23.0,1.0,Учитель
2,7717666051,Афанасова Анна Евгеньевна,027-432-474 41,173924.39,23.0,23.0,1.0,Учитель
3,7717666051,Баженов Александр Константинович,138-721-156 63,134256.32,23.0,23.0,1.0,Учитель
4,7717666051,Бакайкина Елена Петровна,049-158-674 93,78585.58,17.0,23.0,1.0,Учитель
...,...,...,...,...,...,...,...,...
56108,7705694400,Шалбурова Ногала Доржиевна,003-917-826 41,104185.50,18.0,18.0,1.0,Учитель
56109,7705694400,Шамаева Галина Ивановна,029-216-255 40,98102.85,18.0,18.0,1.0,Учитель
56110,7705694400,Шестова Сусанна Александровна,052-034-242 00,106681.64,17.0,18.0,1.0,Учитель
56111,7705694400,Шиндлер Инна Николаевна,124-358-525 51,140290.89,16.0,18.0,1.0,Учитель


In [30]:
info.dropna()

,inn,fio,snils,sum_may,wday_may,nday_may,stv_may,job_may
0,7717666051,Алимова Елена Владимировна,114-535-773 47,174937.67,23.0,23.0,1.0,Учитель
1,7717666051,Архипова Яна Николаевна,130-996-942 92,121383.16,23.0,23.0,1.0,Учитель
2,7717666051,Афанасова Анна Евгеньевна,027-432-474 41,173924.39,23.0,23.0,1.0,Учитель
3,7717666051,Баженов Александр Константинович,138-721-156 63,134256.32,23.0,23.0,1.0,Учитель
4,7717666051,Бакайкина Елена Петровна,049-158-674 93,78585.58,17.0,23.0,1.0,Учитель
...,...,...,...,...,...,...,...,...
56108,7705694400,Шалбурова Ногала Доржиевна,003-917-826 41,104185.50,18.0,18.0,1.0,Учитель
56109,7705694400,Шамаева Галина Ивановна,029-216-255 40,98102.85,18.0,18.0,1.0,Учитель
56110,7705694400,Шестова Сусанна Александровна,052-034-242 00,106681.64,17.0,18.0,1.0,Учитель
56111,7705694400,Шиндлер Инна Николаевна,124-358-525 51,140290.89,16.0,18.0,1.0,Учитель


In [34]:
def give_true_sum(row):
    ans = row['sum_' + month]
    if row['stv_' + month] < 1:
        ans /= row['stv_' + month]
    if row['wday_' + month] < row['nday_' + month]:
        ans *=  row['nday_' + month] / row['wday_' + month]
    return ans

In [35]:
info['true_sum'] = info.apply(lambda row: give_true_sum(row), axis=1)

In [38]:
info[info.snils == '110-142-290 75']

,inn,fio,snils,sum_may,wday_may,nday_may,stv_may,job_may,true_sum
5682,7728007477,Павлова Жанна Александровна,110-142-290 75,58839.52,17.0,18.0,0.89,Учитель,70000.750826


In [37]:
print_df(info[info.true_sum < 94100], '84- после нормализации и мэш')

In [32]:
info[info.sum_may < 94100]

,inn,fio,snils,sum_may,wday_may,nday_may,stv_may,job_may
4,7717666051,Бакайкина Елена Петровна,049-158-674 93,78585.58,17.0,23.0,1.0,Учитель
28,7717666051,Демидова Наталья Владимировна,056-837-897 20,93470.67,23.0,23.0,1.0,Учитель
52,7717666051,Мошкина Мария Александровна,130-260-884 18,84132.00,23.0,23.0,1.0,Учитель
58,7717666051,Потапова Татьяна Алексеевна,018-573-409 61,84173.85,23.0,23.0,1.0,Учитель
69,7717666051,Смирнова Екатерина Владимировна,072-736-670 82,86325.10,23.0,23.0,1.0,Учитель
...,...,...,...,...,...,...,...,...
56076,7705694400,Рожайская Мария Владимировна,130-548-355 38,84210.83,18.0,18.0,1.0,Учитель
56077,7705694400,Рубцова Наталья Вадимовна,016-228-614 27,94097.93,18.0,18.0,1.0,Учитель
56082,7705694400,Самуйлова Елена Николаевна,017-703-417 31,93057.31,17.0,18.0,1.0,Учитель
56098,7705694400,Тимофеев Андрей Петрович,145-305-904 44,93106.76,17.0,18.0,1.0,Учитель
